In [65]:
from google.colab import drive

drive.mount("/content/gdrive")
!pwd  # show current path

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Reto IA Periodo2/Datos/Fact


In [66]:
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Base"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Base
20230223_productos.csv	customers_sampled.csv  ventas.csv


In [67]:
import pandas as pd
tCliente = pd.read_csv("customers_sampled.csv")
tProducto = pd.read_csv("20230223_productos.csv")
tVenta = pd.read_csv("ventas.csv")
#print(tClientes)

In [68]:
tVenta["calmonth"] = pd.to_datetime( tVenta["calmonth"].astype(str) + "01" )

tCliente = tCliente.astype({'CustomerId': 'int'})
tVenta = tVenta.astype({'CustomerId': 'int', 'material': "int"})
tVenta["calmonth"] = pd.to_datetime(tVenta["calmonth"])
tProducto = tProducto.astype({'Material': "int"})


In [69]:
tVenta = tVenta.rename(columns={'material': "Material"})

# Limpieza

In [70]:
tVenta.count()

CustomerId    2347110
Material      2347110
calmonth      2347110
uni_box       2347110
dtype: int64

In [71]:
tVenta = tVenta[tVenta["calmonth"] >= "2020-01-01"]

In [72]:
tVenta.count()

CustomerId    2139792
Material      2139792
calmonth      2139792
uni_box       2139792
dtype: int64

## Generacion Productos y Ventas Test

In [73]:
oFirstDate = tVenta.groupby('Material')["calmonth"].min().reset_index()
tProducto = tProducto.merge(oFirstDate, on='Material', how='left')
tProducto.rename(columns={'calmonth': 'dPrimeraVenta'}, inplace=True)

## Clientes sin Compras

In [74]:
tCliente = tCliente.drop( tCliente[~tCliente["CustomerId"].isin(tVenta["CustomerId"])].index )

tCliente.fillna(0, inplace=True)

In [75]:
tCliente[~tCliente["CustomerId"].isin(tVenta["CustomerId"])].index

Index([], dtype='int64')

## Clientes Pocas Ventas

In [76]:
oFirstDate = tVenta.groupby('CustomerId')["calmonth"].min().reset_index()
# Renombra la columna para hacer el merge
oFirstDate.rename(columns={'calmonth': 'dPrimeraCompraCliente'}, inplace=True)
tVenta = tVenta.merge(oFirstDate, on='CustomerId', how='left')
#tVenta["dPrimeraCompraCliente"] = tVenta.groupby('CustomerId')["calmonth"].min()
#tVenta["dPrimeraCompraCliente"] =
#tVenta
tVentaGroup = tVenta.groupby(["CustomerId", 'dPrimeraCompraCliente'])["uni_box"].sum().reset_index()

tClientesVentaRecientes = tVentaGroup[tVentaGroup["dPrimeraCompraCliente"] >=  "2022-09-01"  ].sort_values(by="uni_box", ascending=False)

tCliente = tCliente.drop( tCliente[tCliente["CustomerId"].isin(tClientesVentaRecientes["CustomerId"])].index )



## Productos no Relacionados

In [77]:
tVenta = tVenta.drop( tVenta[~tVenta["Material"].isin(tProducto["Material"])].index )
tProductoNoVendido = tProducto[~tProducto["Material"].isin(tVenta["Material"])]
tProducto = tProducto.drop( tProducto[~tProducto["Material"].isin(tVenta["Material"])].index )

# Dimensiones Clientes


## dimZona

In [78]:
oData = {"iIdZona": [1,2,3,4,5,6,7,8,9,10], "sZona": ["Agricola", "Comercial", "General", "Habitacional", "Habitacional Mixta", "Industrial", "Minero", "Mixta", "Negocios", "Turismo"]}

dimZona = pd.DataFrame().from_dict(oData)
print(dimZona)
oData = []

   iIdZona               sZona
0        1            Agricola
1        2           Comercial
2        3             General
3        4        Habitacional
4        5  Habitacional Mixta
5        6          Industrial
6        7              Minero
7        8               Mixta
8        9            Negocios
9       10             Turismo


## dimNivelSocioEconomico

In [79]:
oData = {"iIdNivelSocioeconomico": [1,2,3,4], "sNivelSocioeconomico": ["A/B", "C", "D", "E"]}

dimNivelSocioEconomico = pd.DataFrame().from_dict(oData)
print(dimNivelSocioEconomico)

   iIdNivelSocioeconomico sNivelSocioeconomico
0                       1                  A/B
1                       2                    C
2                       3                    D
3                       4                    E


## dimSubCanal

In [80]:
dimSubCanal = pd.DataFrame()
dimSubCanal = tCliente.groupby('sub_canal_comercial').size().reset_index()
dimSubCanal.columns = ['sSubCanal', 'iCount']
dimSubCanal.drop(columns="iCount", inplace= True)

dimSubCanal["iIdSubCanal"] = dimSubCanal.index + 1

dimSubCanal = dimSubCanal[['iIdSubCanal','sSubCanal']]
dimSubCanal

iIdSubCanal                                      sSubCanal
0             1      Abarrotes / Almacenes / Bodegas / Víveres
1             2             Carnicería / Pollería / Pescadería
2             3                              Cerveza y Licores
3             4                         Estanquillos / kioscos
4             5                         Farmacia Independiente
5             6                              Frutas y Verduras
6             7                                Hogar con Venta
7             8                           Mayorista Abarrotero
8             9                         Minisuper / Minimarket
9            10                         Panadería / Pastelería
10           11                   TDC/Proximidad Independiente
11           12  Tiendas de Alimentos Especializados Orgánicos
12           13                                    Tortillería

# Transformacion

## Transformación Categorias

In [81]:
tProducto["ProductCategory"].unique()

array(['AGUA', 'CATEGORIAS EN EXPANSION', 'REFRESCOS', 'LACTEOS',
       'AGUA MINERAL CON SABOR', 'BEBIDAS CON ALCOHOL'], dtype=object)

In [82]:

# Actualizar la columna 'ProductCategory' para los cafés
oCafe = ['CAFE MOLIDO', 'CAFE GRANO']
tProducto.loc[tProducto['ProductType'].isin(oCafe), 'ProductCategory'] = 'CAFES'

# Actualizar la columna 'ProductCategory' para jugos y néctares
oJugoNectar = ['JUGOS Y NECTARES', 'TE', 'NARANJADAS', 'BEBIDAS REFRESCANTES']
tProducto.loc[tProducto['ProductType'].isin(oJugoNectar), 'ProductCategory'] = 'JUGOS Y NECTARES'

# Actualizar la columna 'ProductCategory' para bebidas alternativas
oBebidaAlternativa = ['BEBIDAS DE SOYA', 'BEBIDAS ENERGETICAS', 'BEBIDAS INFANTILES']
tProducto.loc[tProducto['ProductType'].isin(oBebidaAlternativa), 'ProductCategory'] = 'BEBIDAS ALTERNATIVAS'

# Actualizar la columna 'ProductCategory' para agua saludable
oAguaSaludable = ['ISOTONICOS', 'AGUA FUNCIONAL', 'BEBIDA CON ELECTROLITOS']
tProducto.loc[tProducto['ProductType'].isin(oAguaSaludable), 'ProductCategory'] = 'AGUA SALUDABLE'

# Reemplazar 'CATEGORIAS EN EXPANSION' si es necesario
tProducto.loc[tProducto['ProductCategory'] == 'CATEGORIAS EN EXPANSION', 'ProductCategory'] = tProducto['ProductCategory'].where(
    ~tProducto['ProductType'].isin(oJugoNectar + oBebidaAlternativa + oAguaSaludable) & (tProducto['ProductType'] != 'Cafe'),
    'JUGOS Y NECTARES'
)




## Dimensiones Productos

### dimProductoTipo

In [83]:
dimProductoTipo = pd.DataFrame()
dimProductoTipo = tProducto.groupby('ProductType').size().reset_index()
dimProductoTipo.columns = ['sProductoTipo', 'iCount']
dimProductoTipo.drop(columns="iCount", inplace= True)

dimProductoTipo["iIdProductoTipo"] = dimProductoTipo.index + 1

dimProductoTipo = dimProductoTipo[['iIdProductoTipo','sProductoTipo']]
dimProductoTipo


iIdProductoTipo            sProductoTipo
0                 1           AGUA FUNCIONAL
1                 2             AGUA MINERAL
2                 3          AGUA PURIFICADA
3                 4          AGUA SABORIZADA
4                 5        BEBIDA ALCOHOLICA
5                 6          BEBIDAS DE SOYA
6                 7      BEBIDAS ENERGETICAS
7                 8       BEBIDAS INFANTILES
8                 9     BEBIDAS REFRESCANTES
9                10               CAFE GRANO
10               11              CAFE MOLIDO
11               12              COLAS LIGHT
12               13            COLAS REGULAR
13               14               ISOTONICOS
14               15         JUGOS Y NECTARES
15               16  LECHE UHT ESPECIALIZADA
16               17        LECHE UHT REGULAR
17               18     LECHE UHT SABORIZADA
18               19                   MIXTOS
19               20               NARANJADAS
20               21            SABORES LIGHT
21               22          SABORES REGULAR
22               23                       TE

### dimContenedor

In [84]:
dimContenedor = pd.DataFrame()
dimContenedor = tProducto.groupby('Container').size().reset_index()
dimContenedor.columns = ['sContenedor', 'iCount']
dimContenedor.drop(columns="iCount", inplace= True)

dimContenedor["iIdContenedor"] = dimContenedor.index + 1

dimContenedor = dimContenedor[['iIdContenedor','sContenedor']]
dimContenedor

iIdContenedor sContenedor
0              1       BOLSA
1              2        LATA
2              3  LATA SLEEK
3              4    PLASTICO
4              5  TETRA PACK
5              6      VIDRIO

### dimTamano

In [85]:
oData = {"iIdTamano": [1,2], "sTamano": ["INDIVIDUAL", "FAMILIAR"]}
dimTamano = pd.DataFrame().from_dict(oData)
print(dimTamano)


   iIdTamano     sTamano
0          1  INDIVIDUAL
1          2    FAMILIAR


### dimProductoCategoria

In [86]:
dimProductoCategoria = pd.DataFrame()
dimProductoCategoria = tProducto.groupby('ProductCategory').size().reset_index()
dimProductoCategoria.columns = ['sProductoCategoria', 'iCount']
dimProductoCategoria.drop(columns="iCount", inplace= True)

dimProductoCategoria["iIdProductoCategoria"] = dimProductoCategoria.index + 1

dimProductoCategoria = dimProductoCategoria[['iIdProductoCategoria','sProductoCategoria']]
dimProductoCategoria


iIdProductoCategoria      sProductoCategoria
0                     1                    AGUA
1                     2  AGUA MINERAL CON SABOR
2                     3          AGUA SALUDABLE
3                     4    BEBIDAS ALTERNATIVAS
4                     5     BEBIDAS CON ALCOHOL
5                     6                   CAFES
6                     7        JUGOS Y NECTARES
7                     8                 LACTEOS
8                     9               REFRESCOS

### dimMarca

In [87]:
dimMarca = pd.DataFrame()
dimMarca = tProducto.groupby('Brand').size().reset_index()
dimMarca.columns = ['sMarca', 'iCount']
dimMarca.drop(columns="iCount", inplace= True)

dimMarca["iIdMarca"] = dimMarca.index + 1

dimMarca = dimMarca[['iIdMarca','sMarca']]
dimMarca


iIdMarca                          sMarca
0          1        ADES ALMENDRA SIN AZUCAR
1          2                     ADES FRUTAL
2          3                     ADES LACTEO
3          4            CIEL AGUA PURIFICADA
4          5                     CIEL EXPRIM
5          6               CIEL MINERALIZADA
6          7                       COCA-COLA
7          8                  COCA-COLA CAFE
8          9                COCA-COLA ENERGY
9         10                 COCA-COLA LIGHT
10        11     COCA-COLA LIGHT SIN CAFEINA
11        12            COCA-COLA SIN AZUCAR
12        13                           COSTA
13        14              COSTA DESCAFEINADO
14        15                       DEL VALLE
15        16             DEL VALLE 100% JUGO
16        17               DEL VALLE RESERVA
17        18                DEL VALLE Y NADA
18        19                  DELAWARE PUNCH
19        20                           FANTA
20        21                    FANTA SENZAO
21        22                       FLORIDA 7
22        23                          FRESCA
23        24                          FRUTSI
24        25                        FUZE TEA
25        26                         GLACEAU
26        27                            JOYA
27        28                    MANZANA LIFT
28        29                  MONSTER ENERGY
29        30                        POWERADE
30        31                    POWERADE FIT
31        32                 PREDATOR ENERGY
32        33                           PULPY
33        34        SANTA CLARA DESLACTOSADA
34        35  SANTA CLARA DESLACTOSADA LIGHT
35        36              SANTA CLARA ENTERA
36        37               SANTA CLARA LIGHT
37        38          SANTA CLARA SABORIZADA
38        39      SANTA CLARA SEMIDESCREMADA
39        40                          SENZAO
40        41                   SIDRAL MUNDET
41        42       SIERRAZUL AGUA PURIFICADA
42        43                          SPRITE
43        44               SPRITE SIN AZUCAR
44        45                     SPRITE ZERO
45        46                 TOPO CHICO A.M.
46        47         TOPO CHICO HARD SELTZER
47        48              TOPO CHICO SANGRIA
48        49        TOPO CHICO SANGRIA LIGHT
49        50                TOPO CHICO TWIST
50        51                      VALLE FRUT

### dimSabor

In [88]:
dimSabor = tProducto.groupby("Flavor").size().reset_index()
dimSabor.columns = ["sSabor", "iCount"]
dimSabor.drop(columns="iCount", inplace=True)
dimSabor["iIdSabor"] = dimSabor.index + 1
dimSabor = dimSabor[["iIdSabor", "sSabor"]]
dimSabor

iIdSabor             sSabor
0          1  100% JUGO MANZANA
1          2  100% JUGO NARANJA
2          3               AGUA
3          4       AGUA MINERAL
4          5           ALMENDRA
..       ...                ...
75        76          UVA MENTA
76        77           VAINILLA
77        78                XXX
78        79         ZERO SUGAR
79        80         ZERO ULTRA

[80 rows x 2 columns]

## Tranformacion Categorias a Dimensiones - Clientes


### Zona

In [89]:
def assign_zone(row):
    # Diccionario para mapear las columnas a IDs numéricos
    oIdZone = {
        'pc_agr_300m': 1,
        'pc_comercial_300m': 2,
        'pc_generales_300m': 3,
        'pc_habitacional_300m': 4,
        'pc_habitacional_mixta_300m': 5,
        'pc_industrial_300m': 6,
        'pc_minero_300m': 7,
        'pc_mixta_300m': 8,
        'pc_negocios_300m': 9,
        'pc_turismo_300m': 10
    }


    # Inicializa las variables
    max_value = -1
    assigned_zone_id = -1
    assigned_zone_title = 'Unknown'

    # Itera sobre las columnas y busca la que tiene el valor máximo
    for column in oIdZone.keys():
        if row[column] > max_value:
            max_value = row[column]
            assigned_zone_id = oIdZone[column]       # Asigna el ID de zona correspondiente

    return assigned_zone_id

#tClienteZonaTemp = pd.merge(tCliente, tClienteZona, left_on='CustomerId', right_on='CustomerId')
# Aplicar la función a cada fila del DataFrame 'customers'
tCliente['iIdZona'] = tCliente.apply(assign_zone, axis=1, result_type='expand')

print(tCliente['iIdZona'].value_counts())
print(tCliente['iIdZona'].isna().sum())

#print(tClientes['sZonaTitulo'].value_counts())
tCliente[tCliente['iIdZona'].isna()]


iIdZona
4     1416
5      284
3       83
8       51
6       50
1       47
2       31
9        4
10       4
7        1
Name: count, dtype: int64
0


<ipython-input-89-db4ef52df36c>:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tCliente['iIdZona'] = tCliente.apply(assign_zone, axis=1, result_type='expand')


Empty DataFrame
Columns: [CustomerId, pc_agr_300m, pc_comercial_300m, pc_generales_300m, pc_habitacional_300m, pc_habitacional_mixta_300m, pc_industrial_300m, pc_minero_300m, pc_mixta_300m, pc_negocios_300m, pc_turismo_300m, mov_domingo, mov_lunes, mov_martes, mov_miercoles, mov_jueves, mov_viernes, mov_sabado, mov_8_00_9_59, mov_10_00_11_59, mov_12_00_13_59, mov_14_00_5_59, mov_16_00_17_59, mov_18_00_19_59, mov_20_00_21_59, Peso_manza, pob_ab_300m, pob_cmas_300m, pob_c_300m, pob_cmen_300m, pob_dmas_300m, pob_d_300m, pob_e_300m, parques, supermercados, hospitales, preescolares, primarias, secundarias, preparatorias, universidades, gimnasios, h_0_2_conteo_personas_300m, h_3_5_conteo_personas_300m, h_6_11_conteo_personas_300m, h_12_14_conteo_personas_300m, h_15_17_conteo_personas_300m, h_18_24_conteo_personas_300m, h_25_29_conteo_personas_300m, h_30_49_conteo_personas_300m, h_50_59_conteo_personas_300m, h_60_mas_conteo_personas_300m, m_0_2_conteo_personas_300m, m_3_5_conteo_personas_300m, m_6_11_conteo_personas_300m, m_12_14_conteo_personas_300m, m_15_17_conteo_personas_300m, m_18_24_conteo_personas_300m, m_25_29_conteo_personas_300m, m_30_49_conteo_personas_300m, m_50_59_conteo_personas_300m, m_60_mas_conteo_personas_300m, modelos_una_persona_conteo_hogares_300m, modelos_dos_personas_conteo_hogares_300m, modelos_tres_mas_conteo_hogares_300m, modelos_hog_fam_conteo_hogares_300m, modelos_hog_no_fam_conteo_hogares_300m, PADRON_HOMBRES_300m, PADRON_MUJERES_300m, PADRON_300m, LISTA_18_HOMBRES_300m, LISTA_18_MUJERES_300m, LISTA_19_HOMBRES_300m, LISTA_19_MUJERES_300m, LISTA_20_24_HOMBRES_300m, LISTA_20_24_MUJERES_300m, LISTA_25_29_HOMBRES_300m, LISTA_25_29_MUJERES_300m, LISTA_30_34_HOMBRES_300m, LISTA_30_34_MUJERES_300m, LISTA_35_39_HOMBRES_300m, LISTA_35_39_MUJERES_300m, LISTA_40_44_HOMBRES_300m, LISTA_40_44_MUJERES_300m, LISTA_45_49_HOMBRES_300m, LISTA_45_49_MUJERES_300m, LISTA_50_54_HOMBRES_300m, LISTA_50_54_MUJERES_300m, LISTA_55_59_HOMBRES_300m, LISTA_55_59_MUJERES_300m, LISTA_60_64_HOMBRES_300m, LISTA_60_64_MUJERES_300m, LISTA_65_Y_MAS_HOMBRES_300m, LISTA_65_Y_MAS_MUJERES_300m, LISTA_HOMBRES_300m, LISTA_MUJERES_300m, LISTA_300m, desocupados_conteo_personas_300m, ocupados_conteo_personas_300m, estudiantes_conteo_personas_300m, ...]
Index: []

[0 rows x 208 columns]

### Nivel Socioeconomico

In [90]:
def assign_socioeconomic_level(row):
    # Diccionario para mapear las categorías socioeconómicas a IDs numéricos
    socioeconomic_id_mapping = {
        'A/B': 1,
        'C': 2,
        'D': 3,
        'E': 4
    }


    # Variables para almacenar el valor máximo y el nivel socioeconómico asignado
    max_value = -1
    assigned_level_id = -1
    assigned_level_title = 'Unknown'

    # Agrupación de columnas por nivel socioeconómico
    ab_columns = ['pob_ab_300m', 'pob_cmas_300m']
    c_columns = ['pob_c_300m', 'pob_cmen_300m']
    d_columns = ['pob_dmas_300m', 'pob_d_300m']
    e_columns = ['pob_e_300m']

    # Suma de valores por grupo
    ab_sum = row[ab_columns].sum()
    c_sum = row[c_columns].sum()
    d_sum = row[d_columns].sum()
    e_sum = row[e_columns].sum()

    # Determinar cuál grupo tiene el valor más alto
    if ab_sum > max_value:
        max_value = ab_sum
        assigned_level_id = socioeconomic_id_mapping['A/B']

    if c_sum > max_value:
        max_value = c_sum
        assigned_level_id = socioeconomic_id_mapping['C']

    if d_sum > max_value:
        max_value = d_sum
        assigned_level_id = socioeconomic_id_mapping['D']

    if e_sum > max_value:
        max_value = e_sum
        assigned_level_id = socioeconomic_id_mapping['E']

    return assigned_level_id

#tClienteNivelSocieconomicoDemograficoTemp = pd.merge(tCliente, tClienteNivelSocieconomicoDemografico, left_on='CustomerId', right_on='CustomerId')

# Aplicar la función a cada fila del DataFrame 'customers'
tCliente['iIdNivelSocieconomico'] = tCliente.apply(assign_socioeconomic_level, axis=1, result_type='expand')

print(tCliente['iIdNivelSocieconomico'].value_counts())

iIdNivelSocieconomico
3    916
2    690
1    262
4    103
Name: count, dtype: int64


<ipython-input-90-5221b857b5a0>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tCliente['iIdNivelSocieconomico'] = tCliente.apply(assign_socioeconomic_level, axis=1, result_type='expand')


### Sub Canal

In [91]:
tCliente['iIdSubCanal'] = tCliente['sub_canal_comercial'].map(dimSubCanal.set_index('sSubCanal')['iIdSubCanal'])
tCliente.drop(columns="sub_canal_comercial", inplace= True)

<ipython-input-91-cb14e581b281>:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tCliente['iIdSubCanal'] = tCliente['sub_canal_comercial'].map(dimSubCanal.set_index('sSubCanal')['iIdSubCanal'])


## Transformacion Categorias a Dimensiones - Productos

In [92]:
tProducto['iIdProductoTipo'] = tProducto['ProductType'].map(dimProductoTipo.set_index('sProductoTipo')['iIdProductoTipo'])
tProducto.drop(columns="ProductType", inplace= True)

In [93]:
tProducto['iIdContenedor'] = tProducto['Container'].map(dimContenedor.set_index('sContenedor')['iIdContenedor'])
tProducto.drop(columns="Container", inplace= True)

In [94]:
tProducto['iIdMarca'] = tProducto['Brand'].map(dimMarca.set_index('sMarca')['iIdMarca'])
tProducto.drop(columns="Brand", inplace= True)

In [95]:
tProducto['iIdProductoCategoria'] = tProducto['ProductCategory'].map(dimProductoCategoria.set_index('sProductoCategoria')['iIdProductoCategoria'])
tProducto.drop(columns="ProductCategory", inplace= True)

In [96]:
tProducto['iIdTamano'] = tProducto['Size'].map(dimTamano.set_index('sTamano')['iIdTamano'])
tProducto.drop(columns="Size", inplace= True)

In [97]:
tProducto['iIdSabor'] = tProducto['Flavor'].map(dimSabor.set_index('sSabor')['iIdSabor'])
tProducto.drop(columns="Flavor", inplace= True)

In [98]:
tProducto["bReturnability"] = tProducto["Returnability"].map({"RETORNABLE": 1, "NO RETORNABLE": 0})
tProducto.drop(columns="Returnability", inplace= True)

## Transformacion Productos Exitosos Generales

In [99]:
tProducto["bViejo"] = tProducto["dPrimeraVenta"].apply(lambda x: 1 if pd.to_datetime(x) < pd.to_datetime("2021-01-01") else 0)

# Filtrar solo los productos viejos
tProductoViejo = tProducto[ (tProducto["bViejo"] == 1) | (tProducto["iIdProductoCategoria"] == 6) ] #Se agrega tambien la categoria 6 ya que en los productos viejos no existe

productos_viejos = tVenta[tVenta['Material'].isin(tProductoViejo["Material"])]

ventas_promedio_por_mes = productos_viejos.groupby(['Material', "calmonth"])['uni_box'].mean().reset_index()

tProductoVentaPromedioporMes = ventas_promedio_por_mes.groupby("Material")["uni_box"].mean().reset_index()


Se obtienen los primeros 5 meses de cada producto y se saca su media de venta de unidades de cada mes

In [100]:
tProductoMerge = tProducto.merge(tProductoVentaPromedioporMes, on="Material" )
tProductoMerge.rename(columns={'uni_box': 'fVentaPromedio'}, inplace=True)

tProductoViejo = tProductoMerge[(tProductoMerge["bViejo"] == 1) | (tProductoMerge["iIdProductoCategoria"] == 6)]
factProductoCategoriaVentaPromedio = tProductoViejo.groupby("iIdProductoCategoria")["fVentaPromedio"].mean().reset_index()

tProductoMerge.drop(columns=["fVentaPromedio"], inplace=True)

tVentaGroup = tVenta.groupby(["Material", "calmonth"])["uni_box"].mean().reset_index()

tProductoVentaMerge = tVentaGroup.merge(tProducto, on="Material")
tProductoVenta_inicio = tProductoVentaMerge[(tProductoVentaMerge["calmonth"] >= tProductoVentaMerge["dPrimeraVenta"]) & ( pd.to_datetime( tProductoVentaMerge["calmonth"] ) <= ( pd.to_datetime( tProductoVentaMerge["dPrimeraVenta"] ) + pd.DateOffset(months=5)) ) ]

#Se agrupa Material y Categoria de producto de los productos viejos para sacar ventas de unidades por los primeros 5 meses
tProductoVenta_inicioGroup = tProductoVenta_inicio.groupby(["Material", "iIdProductoCategoria"])["uni_box"].mean().reset_index()

Con la venta promedio de cada producto se empieza a checar si la venta es mayor o igual a la venta promedio por categoria (Usando los productos viejos como base para el indice) y se especifica si es un producto de exito o no

In [101]:
#Funcion lambda para sacar la venta promedio por la categoria que se seleccione
funcVentaPromedioCategoria = lambda x : factProductoCategoriaVentaPromedio[factProductoCategoriaVentaPromedio["iIdProductoCategoria"] == x]["fVentaPromedio"]

#tProductoVenta_inicioGroup["fVentaPromedioCategoria"] = tProductoVenta_inicioGroup["iIdProductoCategoria"].apply(funcVentaPromedioCategoria.values[0])

#Agregamos la columna de Exitoso
tProductoVenta_inicioGroup["bExitoso"] = 0

#Iteramos sobre cada fila y checamos si el promedio de venta de los primeros 5 meses es mayor a la venta promedio de cada categoria
for index, row in tProductoVenta_inicioGroup.iterrows():
  fVentaPromedio = funcVentaPromedioCategoria(row["iIdProductoCategoria"]).values[0]
  if(fVentaPromedio <= row["uni_box"]):
    tProductoVenta_inicioGroup.loc[index, "bExitoso"] = 1

#Limpiamos columnas y generamos la nueva tabla de Productos con la columna de exitoso
tProductoVenta_inicioGroup = tProductoVenta_inicioGroup.drop("iIdProductoCategoria",axis=1)
tProductoNew = tProducto.merge(tProductoVenta_inicioGroup, on="Material")
tProductoNew = tProductoNew.drop("uni_box", axis=1)

tProducto = tProductoNew


# Productos Exitosos Por Cliente

In [102]:
tVentaGroup = tVenta.groupby(["CustomerId","Material", "calmonth"])["uni_box"].mean().reset_index()
tProductoVentaMerge = tVentaGroup.merge(tProducto, on="Material")
tProductoVenta_inicio = tProductoVentaMerge[(tProductoVentaMerge["calmonth"] >= tProductoVentaMerge["dPrimeraVenta"]) & ( pd.to_datetime( tProductoVentaMerge["calmonth"] ) <= ( pd.to_datetime( tProductoVentaMerge["dPrimeraVenta"] ) + pd.DateOffset(months=5)) ) ]
tProductoVenta_inicio = tProductoVenta_inicio.sort_values(by=["CustomerId", "calmonth"], ascending=True)

tProductoFiltro = tProductoVenta_inicio.groupby(["CustomerId", "Material"])["Material"].count().reset_index(name="count")
tProductoFiltro = tProductoFiltro[tProductoFiltro["count"] >= 5]

tProductoClienteExitoso = tProductoVenta_inicio.merge(tProductoFiltro, on=["CustomerId", "Material"], how="right")
tProductoClienteExitoso = tProductoClienteExitoso.groupby(["CustomerId", "Material"])["Material"].count().reset_index(name="count")
tProductoClienteExitoso.drop(columns=["count"], inplace=True)

# Separacion Productos Test

In [103]:
tVentaTest = tVenta[tVenta["calmonth"] > "2022-06-01"]
tProductoTest = tProducto[tProducto["dPrimeraVenta"] > '2022-06-01']

tVenta = tVenta[tVenta["calmonth"] <= "2022-06-01"]
tProducto = tProducto[tProducto["dPrimeraVenta"] <= '2022-06-01']

# Separacion Columnas Clientes

## Zona 300m

In [104]:
oColumns = [
    'CustomerId', 'pc_agr_300m', 'pc_comercial_300m', 'pc_generales_300m',
    'pc_habitacional_300m', 'pc_habitacional_mixta_300m',
    'pc_industrial_300m', 'pc_minero_300m', 'pc_mixta_300m',
    'pc_negocios_300m', 'pc_turismo_300m'
]

# Crear un nuevo DataFrame solo con esas columnas
tClienteZona = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)
# Guardar el nuevo DataFrame en un archivo CSV
#tClienteFiltered.to_csv('customers_movimientos_d.csv', index=False)

oColumns = []

## Nivel Socioeconomico 300m

In [105]:
oColumns = [
    'CustomerId', 'pob_ab_300m', 'pob_cmas_300m', 'pob_c_300m',
    'pob_cmen_300m', 'pob_dmas_300m', 'pob_d_300m',
    'pob_e_300m'
]

# Crear un nuevo DataFrame solo con esas columnas
tClienteNivelSocieconomicoDemografico = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)
# Guardar el nuevo DataFrame en un archivo CSV
#tClienteFiltered.to_csv('customers_movimientos_d.csv', index=False)

oColumns = []

## Movilidad por Dias

In [106]:
oColumns = [
    'CustomerId', 'mov_domingo', 'mov_lunes', 'mov_martes', 'mov_miercoles', 'mov_jueves', 'mov_viernes', 'mov_sabado',
]

# Crear un nuevo DataFrame solo con esas columnas
tClienteMovilidadDia = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)
# Guardar el nuevo DataFrame en un archivo CSV
#tClienteFiltered.to_csv('customers_movimientos_d.csv', index=False)

oColumns = []

## Movilidad por Hora

In [107]:
oColumns = [
    'CustomerId','mov_8_00_9_59', 'mov_10_00_11_59', 'mov_12_00_13_59', 'mov_14_00_5_59', 'mov_16_00_17_59', 'mov_18_00_19_59', 'mov_20_00_21_59'
]

tClienteMovilidadHora = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []

## Conteo Personas por Edad 300m

In [108]:
oColumns = [
    'CustomerId','h_0_2_conteo_personas_300m', 'h_3_5_conteo_personas_300m', 'h_6_11_conteo_personas_300m',
    'h_12_14_conteo_personas_300m', 'h_15_17_conteo_personas_300m', 'h_18_24_conteo_personas_300m',
    'h_25_29_conteo_personas_300m', 'h_30_49_conteo_personas_300m', 'h_50_59_conteo_personas_300m',
    'h_60_mas_conteo_personas_300m', 'm_0_2_conteo_personas_300m', 'm_3_5_conteo_personas_300m',
    'm_6_11_conteo_personas_300m', 'm_12_14_conteo_personas_300m', 'm_15_17_conteo_personas_300m',
    'm_18_24_conteo_personas_300m', 'm_25_29_conteo_personas_300m', 'm_30_49_conteo_personas_300m',
    'm_50_59_conteo_personas_300m', 'm_60_mas_conteo_personas_300m'
]

tClienteEdadDemografica = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

# Guardar el nuevo DataFrame en un archivo CSV
#filtered_customers.to_csv('customers_conteo_personas.csv', index=False)

oColumns = []


## Conteo Personas por Hogares 300m

In [109]:
oColumns = [
    'CustomerId', 'modelos_una_persona_conteo_hogares_300m', 'modelos_dos_personas_conteo_hogares_300m',
    'modelos_tres_mas_conteo_hogares_300m', 'modelos_hog_fam_conteo_hogares_300m',
    'modelos_hog_no_fam_conteo_hogares_300m'
]

tClienteHogaresDemografica = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []

## Conteo Votantes Registrados 300m

In [110]:
oColumns = [
    'CustomerId','LISTA_HOMBRES_300m',  'LISTA_MUJERES_300m', 'LISTA_300m',
    'LISTA_18_HOMBRES_300m', 'LISTA_18_MUJERES_300m', 'LISTA_19_HOMBRES_300m', 'LISTA_19_MUJERES_300m',
    'LISTA_20_24_HOMBRES_300m', 'LISTA_20_24_MUJERES_300m', 'LISTA_25_29_HOMBRES_300m', 'LISTA_25_29_MUJERES_300m',
    'LISTA_30_34_HOMBRES_300m', 'LISTA_30_34_MUJERES_300m', 'LISTA_35_39_HOMBRES_300m', 'LISTA_35_39_MUJERES_300m',
    'LISTA_40_44_HOMBRES_300m', 'LISTA_40_44_MUJERES_300m', 'LISTA_45_49_HOMBRES_300m', 'LISTA_45_49_MUJERES_300m',
    'LISTA_50_54_HOMBRES_300m', 'LISTA_50_54_MUJERES_300m', 'LISTA_55_59_HOMBRES_300m', 'LISTA_55_59_MUJERES_300m',
    'LISTA_60_64_HOMBRES_300m', 'LISTA_60_64_MUJERES_300m', 'LISTA_65_Y_MAS_HOMBRES_300m', 'LISTA_65_Y_MAS_MUJERES_300m'
]

tClienteVotantesDemografica = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []


## Ocupacion 300m

In [111]:
oColumns = [
    'CustomerId', 'desocupados_conteo_personas_300m', 'ocupados_conteo_personas_300m', 'estudiantes_conteo_personas_300m',
    'discapacitados_conteo_personas_300m', 'pensionados_conteo_personas_300m', 'asalariados_conteo_personas_300m',
    'no_remunerados_conteo_personas_300m'
]

tClienteOcupacionDemografica = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []


## Ingresos Mensuales Promedio 300m

In [112]:
oColumns = [
    'CustomerId', 'ingreso_minimo_300m', 'ingreso_promedio_300m', 'ingreso_maximo_300m', 'ingreso_remesas_300m', 'ingreso_rentas_300m'
]

tClienteIngresoDemografico = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []

## Gastos Mensuales Promedio 300m

In [113]:
oColumns = [
    'CustomerId', 'pc_gasto_alimentos_300m', 'pc_gasto_educa_espa_300m', 'pc_gasto_limpieza_300m',
    'pc_gasto_personales_300m', 'pc_gasto_salud_300m', 'pc_gasto_transf_gas_300m',
    'pc_gasto_transporte_300m', 'pc_gasto_vesti_calz_300m', 'pc_gasto_vivienda_300m', 'gasto_promedio_300m', 'gasto_total_300m'
]

tClienteGastoDemografico = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []

## Flujo Poblacional 300m

In [114]:
oColumns = [
    'CustomerId', 'flo_sem_ab_300m', 'flo_sem_c_300m',
    'flo_sem_cmas_300m', 'flo_sem_cmen_300m', 'flo_sem_d_300m',
    'flo_sem_dmas_300m', 'flo_sem_e_300m', 'flo_finde_tot_300m',
    'flo_finde_ab_300m', 'flo_finde_c_300m', 'flo_finde_cmas_300m',
    'flo_finde_cmen_300m', 'flo_finde_d_300m', 'flo_finde_dmas_300m',
    'flo_finde_e_300m', 'flo_est_300m', 'flo_ocu_300m', 'flo_otros_300m',
    'flo_sem_0a14_300m', 'flo_sem_15a29_300m', 'flo_sem_30a44_300m',
    'flo_sem_45a64_300m', 'flo_sem_65amas_300m', 'flo_sem_tot_300m'
]

tClienteFlujoDemografico = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []

## Flujo Autos por Hora 300m

In [115]:
oColumns = [
    'CustomerId', 'autos_hora_0', 'autos_hora_1', 'autos_hora_2', 'autos_hora_3',
    'autos_hora_4', 'autos_hora_5', 'autos_hora_6', 'autos_hora_7',
    'autos_hora_8', 'autos_hora_9', 'autos_hora_10', 'autos_hora_11',
    'autos_hora_12', 'autos_hora_13', 'autos_hora_14', 'autos_hora_15',
    'autos_hora_16', 'autos_hora_17', 'autos_hora_18', 'autos_hora_19',
    'autos_hora_20', 'autos_hora_21', 'autos_hora_22', 'autos_hora_23', 'autos_promedio'
]

tClienteAutoHora = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []

## Velocidad por Hora 300m

In [116]:
oColumns = [
    'CustomerId', 'velocidad_hora_0', 'velocidad_hora_1', 'velocidad_hora_2', 'velocidad_hora_3',
    'velocidad_hora_4', 'velocidad_hora_5', 'velocidad_hora_6', 'velocidad_hora_7',
    'velocidad_hora_8', 'velocidad_hora_9', 'velocidad_hora_10', 'velocidad_hora_11',
    'velocidad_hora_12', 'velocidad_hora_13', 'velocidad_hora_14', 'velocidad_hora_15',
    'velocidad_hora_16', 'velocidad_hora_17', 'velocidad_hora_18', 'velocidad_hora_19',
    'velocidad_hora_20', 'velocidad_hora_21', 'velocidad_hora_22', 'velocidad_hora_23', 'velocidad_promedio'
]

tClienteAutoVelocidadHora = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []

## Padron 300m

In [117]:
oColumns = [
    'CustomerId', 'PADRON_HOMBRES_300m', 'PADRON_MUJERES_300m', 'PADRON_300m'
]

tClientePadron = tCliente[oColumns]

# Borrar
oColumns.remove('CustomerId')

tCliente.drop(columns=oColumns, inplace=True)

oColumns = []

# Exportacion

In [118]:
tCliente.columns

Index(['CustomerId', 'Peso_manza', 'parques', 'supermercados', 'hospitales',
       'preescolares', 'primarias', 'secundarias', 'preparatorias',
       'universidades', 'gimnasios', 'pea_conteo_personas_300m',
       'pnea_conteo_personas_300m', 'arboles_300m', 'banqueta_300m',
       'prob_VPH_TV_300m', 'prob_inter_mod_300m', 'viviendas_300m',
       'POBTOT_300m', 'accesibilidad', 'socioeconomic_status_rgm',
       'industry_customer_size', 'rgm_channel', 'iIdZona',
       'iIdNivelSocieconomico', 'iIdSubCanal'],
      dtype='object')

## Exportacion Caracteristicas Cliente

In [119]:
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos/CaracteristicasCliente"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos/CaracteristicasCliente
tClienteAutoHora.csv		tClienteMovilidadDia.csv
tClienteAutoVelocidadHora.csv	tClienteMovilidadHora.csv
tClienteEdadDemografica.csv	tClienteNivelSocieconomicoDemografico.csv
tClienteFlujoDemografico.csv	tClienteOcupacionDemografica.csv
tClienteGastoDemografico.csv	tClientePadron.csv
tClienteHogaresDemografica.csv	tClienteVotantesDemografica.csv
tClienteIngresoDemografico.csv	tClienteZona.csv


In [120]:
tClienteZona.to_csv('tClienteZona.csv', index=False)
tClienteNivelSocieconomicoDemografico.to_csv('tClienteNivelSocieconomicoDemografico.csv', index=False)
tClienteMovilidadDia.to_csv('tClienteMovilidadDia.csv', index=False)
tClienteMovilidadHora.to_csv('tClienteMovilidadHora.csv', index=False)
tClienteEdadDemografica.to_csv('tClienteEdadDemografica.csv', index=False)
tClienteHogaresDemografica.to_csv('tClienteHogaresDemografica.csv', index=False)
tClienteVotantesDemografica.to_csv('tClienteVotantesDemografica.csv', index=False)
tClienteOcupacionDemografica.to_csv('tClienteOcupacionDemografica.csv', index=False)
tClienteIngresoDemografico.to_csv('tClienteIngresoDemografico.csv', index=False)
tClienteGastoDemografico.to_csv('tClienteGastoDemografico.csv', index=False)
tClienteFlujoDemografico.to_csv('tClienteFlujoDemografico.csv', index=False)
tClienteAutoHora.to_csv('tClienteAutoHora.csv', index=False)
tClienteAutoVelocidadHora.to_csv('tClienteAutoVelocidadHora.csv', index=False)
tClientePadron.to_csv('tClientePadron.csv', index=False)

## Exportacion Dimensiones

In [121]:
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Dimensiones"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Dimensiones
dimContenedor.csv  dimNivelSociEconomico.csv  dimProductoTipo.csv  dimSubCanal.csv  dimZone.csv
dimMarca.csv	   dimProductoCategoria.csv   dimSabor.csv	   dimTamano.csv


In [122]:
dimZona.to_csv('dimZone.csv', index=False)
dimNivelSocioEconomico.to_csv('dimNivelSociEconomico.csv', index=False)
dimSubCanal.to_csv('dimSubCanal.csv', index=False)
dimProductoTipo.to_csv('dimProductoTipo.csv', index=False)
dimProductoCategoria.to_csv('dimProductoCategoria.csv', index=False)
dimMarca.to_csv('dimMarca.csv', index=False)
dimTamano.to_csv('dimTamano.csv', index=False)
dimContenedor.to_csv('dimContenedor.csv', index=False)
dimSabor.to_csv('dimSabor.csv', index=False)

## Exportacion Tablas

In [123]:
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos
Base			Dimensiones  Predicciones  Test		  tVenta.csv
CaracteristicasCliente	Fact	     tCliente.csv  tProducto.csv


In [124]:
tCliente.to_csv('tCliente.csv', index=False)
tProducto.to_csv('tProducto.csv', index=False)
tVenta.to_csv('tVenta.csv', index=False)

## Exportacion Test

In [125]:
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Test"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Test
tProductoTest.csv  tVentaTest.csv


In [126]:
tProductoTest.to_csv('tProductoTest.csv', index=False)
tVentaTest.to_csv('tVentaTest.csv', index=False)

## Exportacion Fact

In [127]:
%cd "/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Fact"
!ls  # show current directory

/content/gdrive/MyDrive/Reto IA Periodo2/Datos/Fact
factClienteCompraVariacion.csv		     factClienteVentaCategoria.csv
factClienteCompraVariacionTipo.csv	     factClienteVentaTipo.csv
factClienteCompraVariacionTipoExitoso.csv    tProductoClienteExitoso.csv
factClienteProductoVentaPromedioMensual.csv


In [128]:
tProductoClienteExitoso.to_csv('tProductoClienteExitoso.csv', index=False)